In [1]:
import pandas as pd
import glob
from read_csv_gz import read_csv_gz
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ed_diagnosis_df = read_csv_gz("ed_data/diagnosis.csv.gz")
ed_edstays_df = read_csv_gz("ed_data/edstays.csv.gz")
ed_medrecon_df = read_csv_gz("ed_data/medrecon.csv.gz")
ed_pyxis_df = read_csv_gz("ed_data/pyxis.csv.gz")
ed_triage_df = read_csv_gz("ed_data/triage.csv.gz")
ed_vitalsign_df = read_csv_gz("ed_data/vitalsign.csv.gz")
hosp_admissions_df = read_csv_gz("hosp_data/admissions.csv.gz")
hosp_patients_df = read_csv_gz("hosp_data/patients.csv.gz")

In [3]:
# ed_edstays_df.head(5)

In [4]:
# hosp_admissions_df.head(15)

In [5]:
# Why does hosptial admittime come BEFORE edouttime in most cases?
(hosp_admissions_df['edouttime'] < hosp_admissions_df['admittime']).value_counts()

False    537006
True       9022
Name: count, dtype: int64

In [6]:
# hosp_patients_df.head(10)

In [7]:
def merge_df(df_chef, df_2, df_3):
    """
    Merge the three dataframes together and calculate the patient age at the adimission time to emergency department.
    :param df_chef: the main dataframe, edstays
    :param df_2: the second dataframe, patients, which contains the patient's anchor age and anchor year
    :param df_3: the third dataframe, admissions, which contains the patient's language (Note: insurance and marital status could be different for a given patient thus are not used in this function)
    :return: a new dataframe with the patient's age and language at the admission time to emergency department
    """ 
    df_2_cleaned = df_2[['subject_id', 'anchor_age', 'anchor_year']].drop_duplicates()
    # df_3_cleaned = df_3[['subject_id', 'insurance', 'marital_status', 'language']].drop_duplicates(subset='subject_id')
    df_3_cleaned = df_3[['subject_id', 'language']].drop_duplicates() 
    ans = pd.merge(df_chef, df_3_cleaned, on='subject_id', how='left') 
    ans = pd.merge(ans, df_2_cleaned, on='subject_id', how='left') 
    ans['admission_age'] = pd.to_datetime(ans['intime']).dt.year - ans['anchor_year'] + ans['anchor_age'] # calculate age at admission time
    ans = ans.drop(columns=['anchor_age', 'anchor_year']) # drop anchor_age and anchor_year and only keep admission_age
    return ans


In [8]:
merged_df = merge_df(df_chef=ed_edstays_df, df_2=hosp_patients_df, df_3=hosp_admissions_df)

In [11]:
# merged_df.head(15)

In [ ]:
cols = ['admission_age']
for col in cols:
    print(col, merged_df[col].isnull().sum() / merged_df.shape[0]) # check the percentage of missing values

admission_age 0.00017878693067536764


In [ ]:
# merged_df[merged_df['admission_age'].isnull()] # check the rows with missing values

,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,language,admission_age
478,10010663,22209635.0,38069845,2146-09-28 04:48:00,2146-09-28 23:55:39,F,HISPANIC/LATINO - GUATEMALAN,WALK IN,ADMITTED,NaN,NaN
479,10010663,NaN,32541009,2148-11-16 21:56:00,2148-11-17 03:13:00,F,HISPANIC/LATINO - GUATEMALAN,WALK IN,HOME,NaN,NaN
480,10010663,NaN,34448503,2144-11-19 18:05:00,2144-11-19 21:22:00,F,HISPANIC/LATINO - GUATEMALAN,WALK IN,HOME,NaN,NaN
481,10010663,NaN,37342540,2146-09-27 03:15:00,2146-09-27 08:29:00,F,HISPANIC/LATINO - GUATEMALAN,WALK IN,HOME,NaN,NaN
9958,10246884,20330558.0,30078457,2172-03-24 01:42:00,2172-03-24 08:08:00,M,BLACK/AFRICAN AMERICAN,AMBULANCE,HOME,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
381793,18971568,NaN,30197314,2130-05-05 15:06:00,2130-05-05 19:21:00,F,WHITE,WALK IN,HOME,NaN,NaN
382580,18992882,NaN,39149470,2168-02-25 18:02:00,2168-02-25 21:27:00,M,OTHER,AMBULANCE,HOME,NaN,NaN
393684,19257662,NaN,39920504,2185-10-22 18:36:00,2185-10-22 22:36:00,F,OTHER,WALK IN,HOME,NaN,NaN
423691,19965819,28425094.0,38077665,2164-02-08 04:10:00,2164-02-08 08:06:00,F,WHITE,AMBULANCE,ADMITTED,NaN,NaN


In [17]:
new_merged_df = pd.merge(ed_edstays_df, hosp_admissions_df, on='hadm_id', how='left')
ed_edstays_df['hadm_id'].isnull().sum() / ed_edstays_df.shape[0] # check the percentage of missing values

np.float64(0.5224130589738101)

def merge(df_chef, df_2, df_3):
    # df_1 contains subject_id, df_2 contains age, df_3 contains insurance, languge, 